In [1]:
import pandas as pd
import sqlite3

import requests  # used later to download additional data

# for the progress bar
from tqdm.auto import tqdm
from pathlib import Path

# to measure runtime
from time import perf_counter

- [`pandas` Cheat Sheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)
- [`matplotlib` Cheat Sheet](https://matplotlib.org/cheatsheets/_images/cheatsheets-1.png)
- [SQL Cheat Sheet](https://www.sqltutorial.org/sql-cheat-sheet/)

# Loading the data

In [2]:
df = pd.read_csv('https://drive.switch.ch/index.php/s/UEpTFv2Bfa5C1dd/download')
df.head()

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,...,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
0,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,0,...,0.50,1,1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696
1,john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,0,...,0.75,2,2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504
2,abdon-prats,Abdón Prats,RCD Mallorca,Spain,17.12.1992,181.0,79.0,NaN,1,0,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
3,pablo-mari,Pablo Marí,RCD Mallorca,Spain,31.08.1993,191.0,87.0,Center Back,1,1,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
4,ruben-pena,Rubén Peña,Real Valladolid,Spain,18.07.1991,172.0,70.0,Right Midfielder,1,1,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002


We repeat our simple data cleaning here, by getting rid of all `NaN` values.

In [3]:
df = df.dropna()

We will be using [`sqlalchemy`](https://www.sqlalchemy.org/) here. First we store the data from the DataFrame in a sqlite3 database.

You can find an `sqlalchemy` cheat sheet [here](https://www.pythonsheets.com/notes/python-sqlalchemy.html).

In [4]:
with sqlite3.connect('crowdstorming.db') as c:
    df.to_sql('crowdstorming', c, if_exists="replace")

from sqlalchemy import create_engine, MetaData

engine = create_engine('sqlite:///crowdstorming.db')
metadata = MetaData()
metadata.reflect(engine)

table_names = metadata.tables.keys()
print(table_names)

dict_keys(['countries', 'crowdstorming'])


# Task 3.1 - Loading additional data

To enrich our data we will collect information about the countries. For this we will use an API.

- Make a GET request to https://restcountries.com/v3.1/all. You can use the [`requests` library](https://requests.readthedocs.io/en/latest/user/quickstart/) for this.
- Create a DataFrame called `countries_df` from the response
- Alternative: Load the data from the file `countries.json` (in case the API dies)
- You may need either [`pd.DataFrame.from_records`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.from_records.html), [`pd.read_json`](https://pandas.pydata.org/docs/reference/api/pandas.read_json.html).

In [5]:
with requests.get('https://restcountries.com/v3.1/all') as response:
    response_json = response.json()
    countries_df = pd.DataFrame.from_records(response_json)

countries_df

,name,tld,cca2,ccn3,cca3,cioc,independent,status,unMember,currencies,...,gini,fifa,car,timezones,continents,flags,coatOfArms,startOfWeek,capitalInfo,postalCode
0,"{'common': 'Burkina Faso', 'official': 'Burkin...",[.bf],BF,854,BFA,BUR,True,officially-assigned,True,"{'XOF': {'name': 'West African CFA franc', 'sy...",...,{'2014': 35.3},BFA,"{'signs': ['BF'], 'side': 'right'}",[UTC],[Africa],"{'png': 'https://flagcdn.com/w320/bf.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [12.37, -1.52]}",NaN
1,"{'common': 'Hong Kong', 'official': 'Hong Kong...","[.hk, .香港]",HK,344,HKG,HKG,False,officially-assigned,False,"{'HKD': {'name': 'Hong Kong dollar', 'symbol':...",...,NaN,HKG,"{'signs': ['HK'], 'side': 'left'}",[UTC+08:00],[Asia],"{'png': 'https://flagcdn.com/w320/hk.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [22.267, 114.188]}",NaN
2,"{'common': 'United Arab Emirates', 'official':...","[.ae, امارات.]",AE,784,ARE,UAE,True,officially-assigned,True,{'AED': {'name': 'United Arab Emirates dirham'...,...,{'2018': 26.0},UAE,"{'signs': ['UAE'], 'side': 'right'}",[UTC+04:00],[Asia],"{'png': 'https://flagcdn.com/w320/ae.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,sunday,"{'latlng': [24.47, 54.37]}",NaN
3,"{'common': 'Antigua and Barbuda', 'official': ...",[.ag],AG,028,ATG,ANT,True,officially-assigned,True,"{'XCD': {'name': 'Eastern Caribbean dollar', '...",...,NaN,ATG,"{'signs': ['AG'], 'side': 'left'}",[UTC-04:00],[North America],"{'png': 'https://flagcdn.com/w320/ag.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [17.12, -61.85]}",NaN
4,"{'common': 'Comoros', 'official': 'Union of th...",[.km],KM,174,COM,COM,True,officially-assigned,True,"{'KMF': {'name': 'Comorian franc', 'symbol': '...",...,{'2014': 45.3},COM,"{'signs': ['COM'], 'side': 'right'}",[UTC+03:00],[Africa],"{'png': 'https://flagcdn.com/w320/km.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [-11.7, 43.23]}",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,"{'common': 'New Zealand', 'official': 'New Zea...",[.nz],NZ,554,NZL,NZL,True,officially-assigned,True,"{'NZD': {'name': 'New Zealand dollar', 'symbol...",...,NaN,NZL,"{'signs': ['NZ'], 'side': 'left'}","[UTC-11:00, UTC-10:00, UTC+12:00, UTC+12:45, U...",[Oceania],"{'png': 'https://flagcdn.com/w320/nz.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [-41.3, 174.78]}","{'format': '####', 'regex': '^(\d{4})$'}"
246,"{'common': 'Italy', 'official': 'Italian Repub...",[.it],IT,380,ITA,ITA,True,officially-assigned,True,"{'EUR': {'name': 'Euro', 'symbol': '€'}}",...,{'2017': 35.9},ITA,"{'signs': ['I'], 'side': 'right'}",[UTC+01:00],[Europe],"{'png': 'https://flagcdn.com/w320/it.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [41.9, 12.48]}","{'format': '#####', 'regex': '^(\d{5})$'}"
247,"{'common': 'El Salvador', 'official': 'Republi...",[.sv],SV,222,SLV,ESA,True,officially-assigned,True,"{'USD': {'name': 'United States dollar', 'symb...",...,{'2019': 38.8},SLV,"{'signs': ['ES'], 'side': 'right'}",[UTC-06:00],[North America],"{'png': 'https://flagcdn.com/w320/sv.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [13.7, -89.2]}","{'format': 'CP ####', 'regex': '^(?:CP)*(\d{4}..."
248,"{'common': 'Svalbard and Jan Mayen', 'official...",[.sj],SJ,744,SJM,NaN,False,officially-assigned,False,"{'NOK': {'name': 'krone', 'symbol': 'kr'}}",...,NaN,NaN,"{'signs': ['N'], 'side': 'right'}",[UTC+01:00],[Europe],"{'png': 'https://flagcdn.com/w320/sj.png', 'sv...",{},monday,"{'latlng': [78.22, 15.63]}",NaN


# Task 3.2 - Data Cleaning
The `name` column contains dictionaries. This makes it annoying for us to work with.
Simplify the column by replacing all entries in it with the value in `common` in that dictionary.

*Hint*: You did something very similar in Task 1.2 last week!

In [ ]:
countries_df = pd.read_json(
    "https://drive.switch.ch/index.php/s/x0zUM0seQqigcU1/download"
)
countries_df["name"] = countries_df["name"].map(lambda x: x["common"])
countries_df.head()

# Task 3.3 - Joining DataFrames

Combine the two DataFrames on the `leagueCountry` column. You can use [`pd.merge`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) for this.
For the DataFrame with the countries, you only need the `name` and `fifa` columns.

In [ ]:
pd.merge(
    df,
    countries_df[["name", "fifa"]],
    how="left",
    left_on="leagueCountry",
    right_on="name",
)

Here is how we can do it with SQLAlchemy.
First we save the data from the DataFrame in the database.

In [ ]:
from sqlalchemy.orm import sessionmaker

with sqlite3.connect("./crowdstorming.db") as c:
    countries_df[["name", "fifa", "unMember"]].to_sql(
        "countries", c, if_exists="replace"
    )

metadata = MetaData()
metadata.reflect(engine)
countries_table = metadata.tables["countries"]
crowdstorming_table = metadata.tables["crowdstorming"]
Session = sessionmaker(bind=engine)
session = Session()
results = session.query(countries_table).all()
print(results[:5])

Then we do the join, but with SQLAlchemy.

In [ ]:
results = (
    session
    .query(
        crowdstorming_table.c,
        countries_table.c.fifa
    )
    .join(
        countries_table,
        crowdstorming_table.c.leagueCountry == countries_table.c.name
    )
    .all()
)
len(results)

# Task 4 - Joining crowdstorming data and country data with SQL

Select all columns from the `crowdstorming` table, and `fifa` and `name` columns from the `countries` table.
Then join the two tables on the `leagueCountry` column of the `crowdstorming` table and the `name` column of the `countries` table.

What JOIN do you need to replicate the results of task 3.3?

First we save the data from the DataFrame in the database.

In [ ]:
with sqlite3.connect("./crowdstorming.db") as c:
    countries_df[["name", "fifa", "unMember"]].to_sql(
        "countries", c, if_exists="replace"
    )


Then we can do the query.


In [ ]:
with sqlite3.connect("./crowdstorming.db") as c:
    result = pd.read_sql(
        """
        SELECT crowdstorming.*, countries.name, countries.fifa
        FROM crowdstorming
        LEFT OUTER JOIN countries ON crowdstorming.leagueCountry = countries.name
        """,
        c,
    )
result

And here is how we can do the same with SQLAlchemy.

In [ ]:
results = (
    session
    .query(
        crowdstorming_table.c,
        countries_table.c.name,
        countries_table.c.fifa,
    )
    .join(
        countries_table,
        crowdstorming_table.c.leagueCountry == countries_table.c.name,
        isouter=True
    )
    .all()
)
print(results[0])

# Task 5 - Calculating the mean

Calculate the mean height and weight of each player in the database.

*Hint*: Check the SQL Aggregate Functions sections in the sidebar in the [cheat sheet](https://www.sqltutorial.org/sql-cheat-sheet/).

In [ ]:
with sqlite3.connect("./crowdstorming.db") as c:
    result = pd.read_sql(
        """
        SELECT AVG(height), AVG(weight)
        FROM crowdstorming
        """,
        c
    )
result

Now repeat this, but on the DataFrame. Are the results the same?

In [ ]:
df[['height', 'weight']].mean()

And the solution with SQLAlchemy:

In [ ]:
from sqlalchemy import func
# calculate the mean height and weight in the database
results = session.query(
    func.avg(crowdstorming_table.c.height),
    func.avg(crowdstorming_table.c.weight)
).all()

print(results)

# Task 6 - Calculating the mean per position

Calculate the mean height and weight of each player per position in the database.

*Hint*: Remember how to do this in pandas. Then check the [cheat sheet](https://www.sqltutorial.org/sql-cheat-sheet/) if there is something similar in SQL.

In [ ]:
with sqlite3.connect("./crowdstorming.db") as c:
    result = pd.read_sql(
        """
        SELECT position, AVG(height), AVG(weight)
        FROM crowdstorming
        GROUP BY position
        """,
        c
    )
result

Now do the same with the DataFrame. Are the results the same?

In [ ]:
df.groupby('position')[['height', 'weight']].mean()

And the solution with SQLAlchemy:

In [ ]:
# calculate mean height and weight PER POSITION in the database
results = (
    session.query(
        crowdstorming_table.c.position,
        func.avg(crowdstorming_table.c.height),
        func.avg(crowdstorming_table.c.weight)
    )
    .group_by(crowdstorming_table.c.position)
    .all()
)
results

# Task 7 - Calculating the mean per position and league
Calculate the mean height and weight of each player per position and per league in the database.

*Hint*: This is almost identical to task 6. Try to not overcomplicate things: What would be the most intuitive way to extend the solution of task 6 to two conditions? (Task 6: 'per position', here: 'per position **and per league**'.)

In [ ]:
with sqlite3.connect("./crowdstorming.db") as c:
    results = pd.read_sql(
        """
        SELECT position, leagueCountry, AVG(height), AVG(weight)
        FROM crowdstorming
        GROUP BY position, leagueCountry
        """,
        c,
    )
results = results.set_index(["position", "leagueCountry"])
results

Now do the same with the DataFrame. Are the results the same?

In [ ]:
df.groupby(['position', 'leagueCountry'])[['height', 'weight']].mean()

And the solution with SQLAlchemy:

In [ ]:
# calculate mean height and weight PER POSITION and PER LEAGUE in the database
results = (
    session
    .query(
        crowdstorming_table.c.position,
        crowdstorming_table.c.leagueCountry,
        func.avg(crowdstorming_table.c.height),
        func.avg(crowdstorming_table.c.weight)
    )
    .group_by(crowdstorming_table.c.position, crowdstorming_table.c.leagueCountry)
    .all()
)
results

# Task 8.1  - People with unusual names
Select all people, whose first name starts with an X, from `people_database.db`.

*Hint*: If you can't figure out how to do this, check [here](https://www.w3schools.com/sql/sql_like.asp).

In [ ]:
people_db_file = Path("people_database.db")
if people_db_file.exists() and people_db_file.stat().st_size == 691134464:
    print("File already downloaded.")
else:
    print("Will download database")
    with requests.get("https://drive.switch.ch/index.php/s/OIsWhbxdTY6h5n7/download", stream=True) as response:
        response.raise_for_status()
        with people_db_file.open('wb') as fo:
            for chunk in tqdm(response.iter_content(chunk_size=8192), desc="Chunks written"):
                fo.write(chunk)

In [ ]:
start = perf_counter()

with sqlite3.connect(people_db_file) as c:
    results = pd.read_sql(
        """
        SELECT * FROM people
        WHERE first_name LIKE 'X%'
        """,
        c,
    )
end = perf_counter()
print(f"Duration: {end - start:.3f}s")
results

Repeat this, but load the data into a DataFrame first, and time both loading it into a DataFrame, and the actual querying.

**Beware, that this will load the entire DB into memory.**

In [ ]:
# alternative: load into dataframe, do it there
start = perf_counter()

with sqlite3.connect(people_db_file) as c:
    people_df = pd.read_sql(
        """
        SELECT * FROM people
        """,
        c,
    )
end = perf_counter()
print(f"Loading duration: {end - start:.3f}s")

start = perf_counter()
result = people_df[people_df["first_name"].str.startswith("X")]
end = perf_counter()
print(f"Just the query: {end - start:.3f}s")
result

# Task 8.2 - Joining with football players
Select all people from `people_database.db`, who share a name with a player from the `crowdstorming` table, as well as the position of that player. Include the `fifa` column from the `countries` table as well for those players, who have a match in that table.

*Hint 1*: You can use the `||` operator to concatenate strings in SQL.

*Hint 2*: You can use the `DISTINCT` keyword to get rid of duplicates.

*Hint 3*: You can have multiple JOINs per query.

*Hint 4*: If you can't solve it in pure SQL, break it down into multiple smaller problems. For example, first get all the names of the players, then join that with the people table.

In [ ]:
with sqlite3.connect(people_db_file) as c:
    c.execute("ATTACH DATABASE './crowdstorming.db' AS crowd")
    results = pd.read_sql(
        """
        SELECT
            DISTINCT
                people.*,
                crowd.crowdstorming.position,
                crowd.countries.fifa
        FROM people
            INNER JOIN
                crowd.crowdstorming
            ON
                people.first_name || ' ' || people.last_name = crowd.crowdstorming.player
            LEFT JOIN
                crowd.countries
            WHERE
                crowd.crowdstorming.leagueCountry = crowd.countries.name
        """,
        c,
    )

results